# Getting started

## Installation

See the [README](https://github.com/ibis-project/ibis-tutorial#setup) for up-to-date installation instructions!

## Download some data

There are other ways to get example data, but we'll start by downloading the penguins dataset
$^1$.



In [ ]:
from pathlib import Path

ddb_file = Path("palmer_penguins.ddb")

if not ddb_file.exists():
    import urllib.request

    urllib.request.urlretrieve(
        "https://storage.googleapis.com/ibis-tutorial-data/palmer_penguins.ddb",
        ddb_file,
    )

DuckDB is similar to sqlite -- we have a single file on disk (or an in-memory connection) that we can operate on.

In [ ]:
import ibis

con = ibis.duckdb.connect("palmer_penguins.ddb", read_only=True)

**Note**: when you connect to a DuckDB database file, DuckDB will create a lock-file to prevent data corruption.  If you see a `palmer_penguins.ddb.wal` file, you can safely ignore it. It will get cleaned up automatically.

In [ ]:
con.list_tables()

In [ ]:
penguins = con.table("penguins")

In [ ]:
penguins

In [ ]:
penguins.head().to_pandas()

In [ ]:
penguins.head().to_pyarrow()

In [ ]:
penguins.head()

## Interactive Mode

In [ ]:
ibis.options.interactive = True

In [ ]:
penguins.head()

## Tables and Columns

`penguins` is a table!  A table is a collection of one or more columns, each with a specific datatype.

In [ ]:
penguins.head()

In [ ]:
type(penguins)

We can look at a single column of that table using the column name as an attribute:

In [ ]:
penguins.species

In [ ]:
type(penguins.species)

## Filter

In [ ]:
penguins.filter(penguins.species == "Adelie")

In [ ]:
penguins.filter((penguins.island == "Dream") & (penguins.species == "Adelie"))

In [ ]:
penguins.filter(
    [
        penguins.island == "Dream",
        penguins.species == "Adelie",
    ]
)

In [ ]:
ibis.to_sql(
    penguins.filter(
    [
        penguins.island == "Dream",
        penguins.species == "Adelie",
    ]
)
)

## Select

In [ ]:
penguins.select("species", "island", "year")

In [ ]:
penguins.select(penguins.species, penguins.island, penguins.year)

In [ ]:
penguins.select("species", "island", penguins.year)

## Drop

In [ ]:
penguins.drop("sex", "year")

In [ ]:
penguins.drop(penguins.sex, penguins.year)

In [ ]:
penguins.drop("sex", penguins.year)

## Mutate

Everything we've seen so far has been subtractive -- removing rows or columns.  What about _adding_ columns?

In [ ]:
penguins.mutate(bill_length_cm=penguins.bill_length_mm / 10)

## Method chaining

You can build up complicated queries by chaining together Ibis methods. The output of many Ibis methods is a table (just like `penguins`!) and we can continue calling table methods until we're satisfied.  Or until we end up with something that _isn't_ a table.  More on that later.

In [ ]:
penguins.select("species", "island", "year", "bill_length_mm").drop("year")

Not the most staggeringly complicated query, but we'll see more soon.

### Exercise 1

Your PI is a cranky American biologist who thinks the metric system is for suckers (oh no).

He demands that we convert all of the existing measurements (`mm` and `g`) to inches and lbs, respectively (I am so sorry). The PI is extra cranky this morning, so we had better make sure that ONLY the silly units are visible in the output.

Some ~helpful~ cursed conversion factors:

|  |  |
| -- | -- |
| mm -> in | divide by 25.4 |
| g -> lb | divide by 453.6 |

Two ways you might accomplish this:
- Chaining `.mutate` to create new columns and `.drop` to drop the old metric columns
- Using a single `.select` to create the new columns as well as select the relevant older columns

Try both ways below! How do they compare?

In [ ]:
# Convert the metric units to imperial, and drop the metric columns.
# Try this using a `.mutate` and `.drop` call.
penguins_imperial_mutate_drop = ...

In [ ]:
# Convert the metric units to imperial, and drop the metric columns.
# Try this using a single `.select` call.
penguins_imperial_select = ...

#### Solutions

In [ ]:
%load solutions/nb01-ex01-mutate-drop.py

In [ ]:
%load solutions/nb01-ex01-select.py

### Does it matter which method you choose?

In this case, no.  Sometimes, there might be a small difference in the generated SQL.

In [ ]:
ibis.to_sql(penguins_imperial_mutate_drop)

In [ ]:
ibis.to_sql(penguins_imperial_select)

But, in practice, this doesn't make a difference.  Any modern SQL execution engine will optimize these two variations to the same set of operations and there will be no measureable performance difference.

## Order By

Want to order your data by a given column or columns?  Use `order_by`!

The default ordering direction is ascending:

In [ ]:
penguins.order_by(penguins.flipper_length_mm)

In [ ]:
penguins.order_by(penguins.flipper_length_mm.desc())

Let's select out a subset of the columns to keep this a bit tidier.

In [ ]:
penguins.order_by(penguins.flipper_length_mm.desc()).select(
    "species", "island", "flipper_length_mm"
)

You can also call `ibis.desc` on the column name to set the order direction:

In [ ]:
penguins.order_by(ibis.desc("flipper_length_mm")).select("species", "island", "flipper_length_mm")

## Aggregate

Ibis has several aggregate functions available to help summarize data.  All the old favorites are there: `mean`, `max`, `min`, `count`, `sum`...

You can aggregate a column by calling the method on that column:

In [ ]:
penguins.flipper_length_mm.mean()

Or you can compute multiple aggregates using the `aggregate` method (also available as `agg` for faster typing):

In [ ]:
penguins.agg([penguins.flipper_length_mm.mean(), penguins.bill_depth_mm.max()])

But aggregates really shine when paired with a `group_by`!

## Group By

`group_by` creates groupings of rows that have the same value for one or more columns.

But it doesn't do much on its own -- you can pair it with aggregate to get a result.

In [ ]:
penguins.group_by("species").agg()

Without any `aggregate` function specified, we get the distinct values of the grouped-column.

We can add a second column to the `group_by` to get the distinct pairs across both columns:

In [ ]:
penguins.group_by(["species", "island"]).agg()

Now, if we add an aggregation function to that, we start to really open things up.

In [ ]:
penguins.group_by(["species", "island"]).agg(penguins.bill_length_mm.mean())

By adding that `mean` to the `aggregate`, we now have a concise way to calculate aggregates over each of the distinct groups in the `group_by`. And we can calculate as many aggregates as we need.

In [ ]:
penguins.group_by(["species", "island"]).agg(
    [penguins.bill_length_mm.mean(), penguins.flipper_length_mm.max()]
)

If we need more specific groups, we can add to the `group_by`.

In [ ]:
penguins.group_by(["species", "island", "sex"]).agg(
        [penguins.bill_length_mm.mean(), penguins.flipper_length_mm.max()]
    )

## Exercises

Time to use what we've learned to answer some penguin questions.

### Exercise 2

What was the largest female penguin (by body mass) on each island in the year 2008?

#### Solution

Note that there are several ways these queries could be written - it's fine if your solution doesn't look like ours, what's important is that the results are the same.

In [ ]:
%load solutions/nb01-ex02.py

### Exercise 3

What was the largest male penguin (by body mass) on each island for each year of data collection?

#### Solution

In [ ]:
%load solutions/nb01-ex03.py

## Some useful extras for staying sane

### `cast`

Sometimes when you parse data, _especially_ from CSVs, the types get a bit messed up. Or you might be loading in a `parquet` file where everything is defined as a `string`.  We can clean that up pretty quickly.

You can cast from floats to ints:

In [ ]:
penguins.bill_length_mm.cast("int32")

And from ints to floats:

In [ ]:
penguins.year.cast("float64")  # this is a terrible idea

You can cast numeric columns to strings:

In [ ]:
penguins.year.cast("str")  # or "string"

And numeric strings to numbers:

In [ ]:
penguins.year.cast("str").cast("int64")

But `ibis` will yell if you try to cast a non-numeric string to a number:

In [ ]:
penguins.species.cast("int32")

### dropna

Does what it says on the box -- drop the `NULL`s from a dataset.

In [ ]:
penguins.dropna()

1: Horst AM, Hill AP, Gorman KB (2020). palmerpenguins: Palmer Archipelago (Antarctica) penguin data. R package version 0.1.0. https://allisonhorst.github.io/palmerpenguins/. doi: 10.5281/zenodo.3960218.